In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 500)

import re

In [2]:
import numpy as np

In [3]:
from geopy.geocoders import Nominatim 
geolocator = Nominatim()

In [4]:
import time
import datetime

In [5]:
df = pd.read_csv("ll_resume_dirty.csv")

In [6]:
#df.head()
#df.dtypes

In [7]:
#df.shape

### Get a rid of duplicates

In [8]:
df.drop_duplicates(subset=["url","content","name","year"], keep="first", inplace = True)

### Take a look at columns that doesn't have data for year

In [9]:
#df[df.year.isnull()]

In [10]:
df["year_clean"] = df["year"].fillna(0)

In [11]:
df.dropna(subset=["content"], inplace = True)

In [12]:
def get_year_from_content(row):
    if row["year_clean"] == 0:
        if "19" in row["content"] or "20" in row["content"]: 
            yr = re.findall("([2019]{2}\d\d)[—－―.-年．，]", row["content"])
            yr = "".join(yr)
        else:
            yr = "unknown"
    else:
        yr = row["year_clean"]
    return yr

In [13]:
df["year_new"] = df.apply(get_year_from_content, axis=1)

In [14]:
df.year_new.replace("unknown", np.nan, inplace = True)

In [15]:
df.dropna(subset=["year_new"], inplace=True)

In [16]:
df.drop(columns = ["year", "year_clean"], inplace = True)

In [17]:
df.year_new.replace("(人民网资料", np.nan, inplace = True)
df.year_new.replace("（人民网资料", np.nan, inplace = True)

In [18]:
df.dropna(subset=["year_new"], inplace=True)

In [19]:
df_new = df

In [20]:
df_new["year_new"] = df_new["year_new"].str.replace("（","")

In [21]:
df_new["year_new"] = df_new["year_new"].str.replace("(","")

In [22]:
df_new["year_new"] = df_new["year_new"].str.replace("其间：","")

In [23]:
df_new["year_new"] = df_new["year_new"].str.replace("期间：","")

In [24]:
df_new["year_new"] = df_new["year_new"].str.replace("l","1")

In [25]:
df_new["year"] = df_new["year_new"].str.extract(r"(^\d{4})").astype(float)

In [26]:
df_new.dropna(subset=["year"], inplace=True)

In [27]:
df_new["year"] = df_new["year"].astype(int)

In [28]:
df_new.drop(columns = ["year_new"], inplace = True)

In [29]:
#df_new.year.value_counts()

### Now onto the content part

#### China has 23 provinces, 4 provincial-level cities, 5 self-governing districts, 2 special districts

北京、上海、天津、重庆

河北省、河南省、湖北省、湖南省、江苏省、江西省、辽宁省、吉林省、黑龙江省、陕西省、山东省、山西省、四川省、青海省、安徽省、海南省、广东省、贵州省、浙江省、福建省、台湾省(central gov counts Taiwan as a province)、甘肃省、云南省

内蒙古自治区、宁夏回族自治区、新疆维吾尔自治区、西藏自治区、广西壮族自治区

香港特别行政区、澳门特别行政区。



## !!! If the content string has none of the above, I'll just drop it, which is not the best thing to do. I'm planning on building a city/town-to-province dictionary, that way in the future I can just convert those strings

In [30]:
df_new["content"] = df_new["content"].str.strip()
df_new["content"] = df_new["content"].str.replace("（", "")
df_new["content"] = df_new["content"].str.replace("(", "")
df_new["content"] = df_new["content"].str.replace("）", "")
df_new["content"] = df_new["content"].str.replace(")", "")

df_new["content"] = df_new["content"].str.replace("；", " ")
df_new["content"] = df_new["content"].str.replace("：", "")
df_new["content"] = df_new["content"].str.replace("—", "")
df_new["content"] = df_new["content"].str.replace("-", "")
df_new["content"] = df_new["content"].str.replace(r"[0-9]{1,}", "")
df_new["content"] = df_new["content"].str.replace(r"[年月]{1,}", "")
df_new["content"] = df_new["content"].str.replace(" ", "")
df_new["content"] = df_new["content"].str.replace("－", "")
df_new["content"] = df_new["content"].str.replace(r"[,，.。]", "")
df_new["content"] = df_new["content"].str.replace(r"[兼任]{1,}", "")
df_new["content"] = df_new["content"].str.replace("挂职", "")
df_new["content"] = df_new["content"].str.replace(r"[期其]间", "")
df_new["content"] = df_new["content"].str.replace(r"[在被]", "")


In [33]:
def get_location(row):
    if "北京" in row["content"]:
        location = "北京"
    elif "上海" in row["content"]:
        location = "上海"
    elif "天津" in row["content"]:
        location = "天津"
    elif "深圳" in row["content"]:
        location = "深圳"
    elif "重庆" in row["content"]:
        location = "重庆"
    elif "河北" in row["content"]:
        location = "河北"
    elif "河南" in row["content"]:
        location = "河南"
    elif "湖北" in row["content"]:
        location = "湖北"        
    elif "湖南" in row["content"]:
        location = "湖南"
    elif "江苏" in row["content"]:
        location = "江苏"    
    elif "江西" in row["content"]:
        location = "江西"
    elif "辽宁" in row["content"]:
        location = "辽宁"
    elif "吉林" in row["content"]:
        location = "吉林"
    elif "黑龙江" in row["content"]:
        location = "黑龙江"
    elif "陕西" in row["content"]:
        location = "陕西"
    elif "黑龙江" in row["content"]:
        location = "黑龙江"
    elif "山东" in row["content"]:
        location = "山东"
    elif "山西" in row["content"]:
        location = "山西"
    elif "四川" in row["content"]:
        location = "四川"
    elif "安徽" in row["content"]:
        location = "安徽"
    elif "青海" in row["content"]:
        location = "青海"
    elif "海南" in row["content"]:
        location = "海南"
    elif "广东" in row["content"]:
        location = "广东"
    elif "贵州" in row["content"]:
        location = "贵州"
    elif "广东" in row["content"]:
        location = "广东"
    elif "浙江" in row["content"]:
        location = "浙江"
    elif "福建" in row["content"]:
        location = "福建"
    elif "台湾" in row["content"]:
        location = "台湾"
    elif "甘肃" in row["content"]:
        location = "甘肃"
    elif "云南" in row["content"]:
        location = "云南"
    elif "内蒙古" in row["content"]:
        location = "内蒙古"
    elif "宁夏" in row["content"]:
        location = "宁夏"
    elif "新疆" in row["content"]:
        location = "新疆维吾尔"
    elif "西藏" in row["content"]:
        location = "西藏"
    elif "广西" in row["content"]:
        location = "广西"
    elif "香港" in row["content"]:
        location = "香港"
    elif "澳门" in row["content"]:
        location = "澳门"
    elif "美国" in row["content"]:
        location = "美国"
    elif "英国" in row["content"]:
        location = "英国"
    elif "加拿大" in row["content"]:
        location = "加拿大"
    elif "法国" in row["content"]:
        location = "法国"
    elif "德国" in row["content"]:
        location = "德国"

    elif "苏州" in row["content"]:
        location = "江苏"
    elif "中央" in row["content"]:
        location = "北京"
    elif "军" in row["content"]:
        location = "解放军"    
    elif "最高" in row["content"]:
        location = "北京"
    elif "部" in row["content"]:
        location = "北京"
    elif "中国" in row["content"]:
        location = "北京"
    elif "国家" in row["content"]:
        location = "北京"
    elif "国务院" in row["content"]:
        location = "北京"
    else:
        location = row["content"][:4]
    return location

In [34]:
df_new["location"] = df_new.apply(get_location, axis=1)

In [331]:
# df_new.location.replace("unknown", np.nan, inplace = True)

In [334]:
# df_new.dropna(subset = ["location"], inplace = True)

In [35]:
def get_geo(row):
    if len(row["location"]) > 3 and row["location"] != "新疆维吾尔":
        address = geolocator.geocode(row["location"], timeout = 5)
        time.sleep(2)
        
        if address == None:
            location = row["location"]
        else:
            address = address.raw["display_name"]
            if "北京" in address:
                location = "北京"
            elif "上海" in address:
                location = "上海"
            elif "天津" in address:
                location = "天津"
            elif "深圳" in address:
                location = "深圳"
            elif "重庆" in address:
                location = "重庆"
            elif "河北" in address:
                location = "河北"
            elif "河南" in address:
                location = "河南"
            elif "湖北" in address:
                location = "湖北"        
            elif "湖南" in address:
                location = "湖南"
            elif "江苏" in address:
                location = "江苏"    
            elif "江西" in address:
                location = "江西"
            elif "辽宁" in address:
                location = "辽宁"
            elif "吉林" in address:
                location = "吉林"
            elif "黑龙江" in address:
                location = "黑龙江"
            elif "陕西" in address:
                location = "陕西"
            elif "黑龙江" in address:
                location = "黑龙江"
            elif "山东" in address:
                location = "山东"
            elif "山西" in address:
                location = "山西"
            elif "四川" in address:
                location = "四川"
            elif "安徽" in address:
                location = "安徽"
            elif "青海" in address:
                location = "青海"
            elif "海南" in address:
                location = "海南"
            elif "广东" in address:
                location = "广东"
            elif "贵州" in address:
                location = "贵州"
            elif "广东" in address:
                location = "广东"
            elif "浙江" in address:
                location = "浙江"
            elif "福建" in address:
                location = "福建"
            elif "台湾" in address:
                location = "台湾"
            elif "甘肃" in address:
                location = "甘肃"
            elif "云南" in address:
                location = "云南"
            elif "内蒙古" in address:
                location = "内蒙古"
            elif "宁夏" in address:
                location = "宁夏"
            elif "新疆" in address:
                location = "新疆维吾尔"
            elif "西藏" in address:
                location = "西藏"
            elif "广西" in address:
                location = "广西"
            elif "香港" in address:
                location = "香港"
            elif "澳门" in address:
                location = "澳门"
            else:
                location = address.split(", ")[-3:]
                location = " ".join(location)
    else:
        location = row["location"]
    return location

## I'll come back to this later

In [36]:
df_clean = df_new

In [37]:
df_clean["location"] = df_clean.apply(get_geo, axis=1)

In [39]:
df_clean.location.value_counts()

北京                                                    3039
重庆                                                    1027
四川                                                     910
河南                                                     777
湖南                                                     689
山东                                                     672
广西                                                     640
湖北                                                     613
广东                                                     609
安徽                                                     592
新疆维吾尔                                                  557
江苏                                                     540
内蒙古                                                    539
甘肃                                                     506
河北                                                     505
黑龙江                                                    500
江西                                                     4

### Add to time series

In [346]:
df_new["year"] = pd.to_datetime(df_new.year,format='%Y')

In [348]:
df_new.head()

,content,name,url,year,location
0,福建省永安县西洋公社插队知青,蔡奇,http://ldzl.people.com.cn/dfzlk/front/personPage16206.htm,1973-01-01,福建
1,福建师范大学政教系政教专业学习,蔡奇,http://ldzl.people.com.cn/dfzlk/front/personPage16206.htm,1975-01-01,福建
2,福建师范大学党委办公室干部,蔡奇,http://ldzl.people.com.cn/dfzlk/front/personPage16206.htm,1978-01-01,福建
3,福建省委办公厅综合处干部、副处长,蔡奇,http://ldzl.people.com.cn/dfzlk/front/personPage16206.htm,1983-01-01,福建
4,福建省委办公厅正处级秘书,蔡奇,http://ldzl.people.com.cn/dfzlk/front/personPage16206.htm,1987-01-01,福建


In [349]:
df_new = df_new.groupby("url").apply(lambda x : x.drop_duplicates("year")
                                          .set_index('year')
                                          .resample('YS')
                                          .ffill())

In [350]:
df_new.head()

content  \
url                                                       year                              
http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm 1989-01-01        河南省新密市财政局办公室主   
                                                          1990-01-01        河南省新密市财政局办公室主   
                                                          1991-01-01        河南省新密市财政局办公室主   
                                                          1992-01-01        河南省新密市财政局办公室主   
                                                          1993-01-01  河南省焱鑫耐火材料厂厂长、党支部副书记   

                                                                     name  \
url                                                       year              
http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm 1989-01-01  高建军   
                                                          1990-01-01  高建军   
                                                          1991-01-01  高建军   
                                                          1992-01-01  高建军   
                                                          1993-01-01  高建军   

                                                                                                                            url  \
url                                                       year                                                                    
http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm 1989-01-01  http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm   
                                                          1990-01-01  http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm   
                                                          1991-01-01  http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm   
                                                          1992-01-01  http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm   
                                                          1993-01-01  http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm   

                                                                     location  
url                                                       year                 
http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm 1989-01-01       河南  
                                                          1990-01-01       河南  
                                                          1991-01-01       河南  
                                                          1992-01-01       河南  
                                                          1993-01-01       河南

In [351]:
df_new = df_new.reset_index(level=["year"])

In [352]:
df_new.head()

,year,content,name,url,location
url,,,,,
http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm,1989-01-01,河南省新密市财政局办公室主,高建军,http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm,河南
http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm,1990-01-01,河南省新密市财政局办公室主,高建军,http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm,河南
http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm,1991-01-01,河南省新密市财政局办公室主,高建军,http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm,河南
http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm,1992-01-01,河南省新密市财政局办公室主,高建军,http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm,河南
http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm,1993-01-01,河南省焱鑫耐火材料厂厂长、党支部副书记,高建军,http://ldzl.people.com.cn/dfzlk/front/personPage10056.htm,河南


In [353]:
df_new.to_csv("ll_resume_clean.csv", index = False)

In [38]:
df[df.name.unique()]

KeyError: "['蔡奇' '李伟' '陈吉宁' ... '梁君彦' '崔世安' '贺一诚'] not in index"